### Import the libraries

In [1]:
import boto3
import pandas as pd
import awswrangler as wr
from io import StringIO
from sagemaker import image_uris, TrainingInput
from sagemaker.estimator import Estimator
import datetime
import boto3
import awswrangler as wr
import pandas as pd
import sagemaker
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.model_monitor import CronExpressionGenerator
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor.model_monitoring import EndpointInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from io import StringIO
import datetime
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
%store

Stored variables and their in-db values:
account_id                                -> '453322325373'
boto_session                              -> '<unavailable>'
bucket_name                               -> 'housing-dataset-8578'
df_test_saved_to_s3                       -> 's3://housing-dataset-8749/processed/housing/test.
df_train_saved_to_s3                      -> 's3://housing-dataset-8749/processed/housing/train
df_validation_saved_to_s3                 -> 's3://housing-dataset-8749/processed/housing/valid
estimator                                 -> '<unavailable>'
feature_store_session                     -> '<unavailable>'
featurestore_runtime                      -> '<unavailable>'
model_name                                -> 'sagemaker-xgboost-2024-06-24-05-19-43'
output_path                               -> 's3://housing-dataset-8749/housing/model'
predictor                                 -> '<unavailable>'
region                                    -> 'us-east-1'
role     

In [3]:
# get bucket_name
%store -r bucket_name
print(bucket_name)

housing-dataset-8578


In [74]:
# save Amazon information
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
s3 = boto3.client('s3', region_name=sagemaker_session.boto_region_name)
# Create the SageMaker Boto3 client
sm_client = boto3.client('sagemaker')

In [5]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### Construct Athena queries to read the data from each offline feature store

In [6]:
train_feature_group_name = "housing_train"
validation_feature_group_name = "housing_validation"
test_feature_group_name = "housing_test"

In [7]:
sagemaker_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'housing_validation',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_validation',
   'CreationTime': datetime.datetime(2024, 6, 26, 3, 11, 39, 364000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_train',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_train',
   'CreationTime': datetime.datetime(2024, 6, 26, 3, 11, 18, 113000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'housing_test',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:453322325373:feature-group/housing_test',
   'CreationTime': datetime.datetime(2024, 6, 26, 3, 12, 31, 583000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': 'cabd8686-40a

In [8]:
# Initialize FeatureGroup
train_feature_group = FeatureGroup(name=train_feature_group_name, sagemaker_session=sagemaker_session)

In [9]:
# Initialize FeatureGroup
validation_feature_group = FeatureGroup(name=validation_feature_group_name, sagemaker_session=sagemaker_session)

In [10]:
# Initialize FeatureGroup
test_feature_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)

In [11]:
train_data_query = train_feature_group.athena_query()
validation_data_query = validation_feature_group.athena_query()
test_data_query = test_feature_group.athena_query()

In [12]:
train_data_table = train_data_query.table_name
validation_data_table = validation_data_query.table_name
test_data_table = test_data_query.table_name

In [13]:
train_query = f"""
SELECT * FROM "{train_data_table}"
"""

In [14]:
validation_query = f"""
SELECT * FROM "{validation_data_table}"
"""

In [15]:
test_query = f"""
SELECT * FROM "{test_data_table}"
"""

### Execute queries and put into dataframes

In [16]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_train = pd.DataFrame()
train_data_query.run(query_string=train_query, output_location='s3://'+bucket_name+'/query_results/train/')
train_data_query.wait()
time.sleep(5)
df_train = train_data_query.as_dataframe()

In [17]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_validation = pd.DataFrame()
validation_data_query.run(query_string=validation_query, output_location='s3://'+bucket_name+'/query_results/validation/')
validation_data_query.wait()
time.sleep(5)
df_validation = validation_data_query.as_dataframe()

In [18]:
# run Athena query. The output is loaded to a Pandas dataframe.
df_test = pd.DataFrame()
test_data_query.run(query_string=test_query, output_location='s3://'+bucket_name+'/query_results/test/')
test_data_query.wait()
time.sleep(5)
df_test = test_data_query.as_dataframe()

### Check the shape of data before dropping unnecessary columns and the saleprice column in the test dataframe

In [19]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(2043, 40)
(584, 40)
(292, 40)


### Get Y values for prediction inference and model quality

In [20]:
df_y_validation = df_validation['saleprice']
df_y_test = df_test['saleprice']

### For df_train and df_validation the "id" column is removed and for df_test both the "id" column and "saleprice" column are removed

In [21]:
df_train = df_train.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_validation = df_validation.drop(['id', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)
df_test = df_test.drop(['id', 'saleprice', 'event_time', 'write_time','api_invocation_time','is_deleted'], axis=1)

### Check the shape of the data

In [22]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(2043, 35)
(584, 35)
(292, 34)


In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mssubclass     2043 non-null   float64
 1   lotarea        2043 non-null   float64
 2   yearbuilt      2043 non-null   float64
 3   yearremodadd   2043 non-null   float64
 4   masvnrarea     2043 non-null   float64
 5   bsmtfinsf1     2043 non-null   float64
 6   bsmtfinsf2     2043 non-null   float64
 7   bsmtunfsf      2043 non-null   float64
 8   totalbsmtsf    2043 non-null   float64
 9   firstflrsf     2043 non-null   float64
 10  secondflrsf    2043 non-null   float64
 11  lowqualfinsf   2043 non-null   float64
 12  grlivarea      2043 non-null   float64
 13  bsmtfullbath   2043 non-null   float64
 14  bsmthalfbath   2043 non-null   float64
 15  fullbath       2043 non-null   float64
 16  halfbath       2043 non-null   float64
 17  bedroomabvgr   2043 non-null   float64
 18  kitchena

### Check the first few rows of the data

In [24]:
df_train.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,20.0,7180.0,1972.0,1972.0,0.0,390.0,0.0,474.0,864.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2008.0,127000.0,11.751943
1,20.0,6000.0,1957.0,1957.0,0.0,308.0,0.0,620.0,928.0,928.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,112500.0,11.630709
2,60.0,13650.0,2002.0,2002.0,232.0,578.0,0.0,1093.0,1671.0,1687.0,...,130.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0,163000.0,12.001506
3,60.0,9720.0,2001.0,2002.0,134.0,1194.0,0.0,163.0,1357.0,1366.0,...,116.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,163000.0,12.001506
4,20.0,8450.0,2000.0,2001.0,108.0,0.0,0.0,1349.0,1349.0,1349.0,...,55.0,0.0,0.0,0.0,0.0,0.0,12.0,2007.0,179000.0,12.095141


In [25]:
df_validation.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,20.0,10481.0,2006.0,2007.0,0.0,0.0,0.0,2140.0,2140.0,2140.0,...,32.0,0.0,0.0,0.0,0.0,0.0,6.0,2007.0,163000.0,12.001506
1,20.0,9717.0,1950.0,1996.0,0.0,49.0,1029.0,0.0,1078.0,1078.0,...,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,142125.0,11.864462
2,60.0,7056.0,1963.0,1963.0,415.0,400.0,0.0,380.0,780.0,983.0,...,50.0,0.0,0.0,0.0,0.0,0.0,10.0,2008.0,140000.0,11.849398
3,30.0,10800.0,1885.0,1995.0,0.0,0.0,0.0,641.0,641.0,1047.0,...,0.0,0.0,0.0,0.0,0.0,450.0,8.0,2007.0,100000.0,11.512925
4,20.0,7000.0,1961.0,1961.0,0.0,781.0,0.0,369.0,1150.0,1150.0,...,0.0,0.0,0.0,162.0,0.0,0.0,7.0,2006.0,163000.0,12.001506


In [26]:
df_test.head(5)

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,logsaleprice
0,20.0,10402.0,2009.0,2009.0,0.0,0.0,0.0,1226.0,1226.0,1226.0,...,0.0,36.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0,12.200558
1,20.0,7024.0,2005.0,2006.0,0.0,980.0,0.0,110.0,1090.0,1090.0,...,0.0,49.0,0.0,0.0,0.0,0.0,0.0,6.0,2007.0,11.964001
2,30.0,10914.0,1929.0,1950.0,0.0,0.0,0.0,715.0,715.0,715.0,...,0.0,0.0,75.0,0.0,112.0,0.0,0.0,8.0,2006.0,12.001506
3,50.0,6000.0,1929.0,1950.0,0.0,0.0,0.0,862.0,862.0,950.0,...,0.0,0.0,112.0,0.0,0.0,0.0,0.0,7.0,2008.0,11.695247
4,50.0,8064.0,1948.0,2004.0,0.0,481.0,174.0,161.0,816.0,816.0,...,414.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0,12.001506


In [27]:
df_train['saleprice'].head(2)

0    127000.0
1    112500.0
Name: saleprice, dtype: float64

In [28]:
df_validation['saleprice'].head(2)

0    163000.0
1    142125.0
Name: saleprice, dtype: float64

### Save the data to the S3 bucket as CSV files

In [29]:
df_train_x_with_header = df_train

In [30]:
df_train_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/train_x_with_header.csv", header=True,  index=False)

In [31]:
df_train.to_csv(f"s3://{bucket_name}/processed/housing/train.csv", header=False,  index=False)

In [32]:
df_train_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

In [33]:
df_validation_x_with_header = df_validation

In [34]:
df_validation_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/validation_x_with_header.csv", header=True, index=False)

In [35]:
df_validation.to_csv(f"s3://{bucket_name}/processed/housing/validation.csv", header=False, index=False)

In [36]:
df_validation_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

In [37]:
df_test_x_with_header = df_test

In [38]:
df_test_x_with_header.to_csv(f"s3://{bucket_name}/processed/housing/test_x_with_header.csv", header=True, index=False)

In [39]:
df_test.to_csv(f"s3://{bucket_name}/processed/housing/test.csv", header=False, index=False)

In [40]:
df_test_saved_to_s3_path = 's3://{}/processed/housing/'.format(bucket_name)

### Train a model using SageMaker built-in XgBoost algorithm on the training data and validate it on the validation data
1. set the container image for the XGBoost algorithm and estimator parameters
2. set the output path and model parameters
3. set the hyperparameters
4. create an estimator
5. fit the model to the training data and validate on the validation data

In [41]:
# Get the current date and time
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Update the model_name variable
model_name = f"sagemaker-xgboost-{current_datetime}"

# set the container image for the XGBoost algorithm
container = image_uris.retrieve(region=region, framework="xgboost", version="latest")

estimator_instance_count = 1

estimator_instance_type = "ml.m5.xlarge"

# Print the model name
print(model_name)

sagemaker-xgboost-2024-06-26-03-54-05


In [42]:
# set the output path
output_path = f"s3://{bucket_name}/housing/model"

# Location in S3 where the model checkpoint will be stored
model_checkpoint_s3_path = '{}/{}/checkpoint/'.format(output_path, model_name)

# Location in S3 where the trained model will be stored
model_output_s3_path = '{}'.format(output_path)



In [43]:
# set the hyperparameters
hyperparameters = {
    "objective": "reg:linear",
    "num_round": "100",
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "silent": "0",
}

In [45]:
# create an estimator
estimator = Estimator(
    image_uri=container,
    role=role,
    instance_count= estimator_instance_count,
    instance_type= estimator_instance_type,
    checkpoint_s3_uri=model_checkpoint_s3_path,    
    output_path=output_path,
    hyperparameters=hyperparameters,
)

In [46]:
# fit the model to the training data and validate on the validation data
estimator.fit(
    {
        "train": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/train.csv", content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=f"s3://{bucket_name}/processed/housing/validation.csv", content_type="text/csv"
        ),
    }
)

INFO:sagemaker:Creating training-job with name: xgboost-2024-06-26-04-00-58-411


2024-06-26 04:00:58 Starting - Starting the training job......
2024-06-26 04:01:36 Starting - Preparing the instances for training...
2024-06-26 04:02:07 Downloading - Downloading input data...
2024-06-26 04:02:32 Downloading - Downloading the training image...
2024-06-26 04:03:23 Training - Training image download completed. Training in progress.
2024-06-26 04:03:23 Uploading - Uploading generated training modelArguments: train
[2024-06-26:04:03:15:INFO] Running standalone xgboost training.
[2024-06-26:04:03:15:INFO] File size need to be processed in the node: 0.49mb. Available memory size in the node: 8136.92mb
[2024-06-26:04:03:15:INFO] Determined delimiter of CSV input is ','
[04:03:15] S3DistributionType set as FullyReplicated
[04:03:15] 2043x34 matrix with 69462 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-06-26:04:03:15:INFO] Determined delimiter of CSV input is ','
[04:03:15] S3DistributionType set as FullyReplicated
[04:03:15] 584x34

### Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [47]:
# Saving training job information to be used in the ML lineage module
training_job_info = estimator.latest_training_job.describe()
if training_job_info != None:
    
    # Get the model data
    model_data = training_job_info["ModelArtifacts"]["S3ModelArtifacts"]
    # Create the primary container
    primary_container = {"Image": container, "ModelDataUrl": model_data}

    # Save our model to the Sagemaker Model Registry
    create_model_response = sagemaker_client.create_model(
        ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
    )
    print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:453322325373:model/sagemaker-xgboost-2024-06-26-03-54-05


In [48]:
# Inspect Training Job Details
training_job_info

{'TrainingJobName': 'xgboost-2024-06-26-04-00-58-411',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:453322325373:training-job/xgboost-2024-06-26-04-00-58-411',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://housing-dataset-8578/housing/model/xgboost-2024-06-26-04-00-58-411/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'reg:linear',
  'silent': '0',
  'subsample': '0.7'},
 'AlgorithmSpecification': {'TrainingImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'train:merror',
    'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:mae',
    'Regex':

### Deploy the model

#### A) Set the deployment parameters

1. Deployment instance details:

    1. Instance count
    
    2. Instance type
    
    3. The Elastic Inference accelerator type
    
2. Serializer and deserializer - set them to CSV as the data will be in CSV format.

In [49]:
# Set the instance count, instance type, endpoint name and other parameters
deploy_initial_instance_count = 1
deploy_instance_type = 'ml.m5.xlarge'
accelerator_type = None
endpoint_name = 'housing-endpoint'
serializer = CSVSerializer(content_type='text/csv')
deserializer = CSVDeserializer(encoding='utf-8', accept='text/csv')

#### B) Set the data capture parameters

Setting up data capture will automatically capture the specified request/response data for calls made to the SageMaker inference endpoint and store them in the specified location in S3.  This is required for the Model Monitor to work.

In [50]:
# Location in S3 that will contain all model monitor related files
model_monitor_s3_path = '{}/{}/model-monitor'.format(output_path, model_name)

# Location in S3 where the data captured from the endpoint will be stored.
# Do not add a '/' at the end of the path here; it will be automatically added by the endpoint. If you add
# a '/' at the end, then the monitoring schedule will not be able to find the data capture files and your
# schedule will fail even if files are present in that S3 location.
data_capture_output_s3_path = '{}/data-capture'.format(model_monitor_s3_path)

# Location in S3 where the results from the baseline job will be stored
data_quality_baseline_job_output_s3_path = '{}/data-quality/baseline/'.format(model_monitor_s3_path)

# Location in S3 where the results from the monitoring job will be stored
# Do not add a '/' at the end of the path here
data_quality_mon_schedule_output_s3_path = '{}/data-quality/monitoring'.format(model_monitor_s3_path)

In [51]:
# Set the data capture configuration
data_capture_config = DataCaptureConfig(
    enable_capture = True,
    sampling_percentage=100,
    destination_s3_uri=data_capture_output_s3_path,
    capture_options=['REQUEST', 'RESPONSE'],
    csv_content_types=['text/csv'],
    json_content_types=None,
    sagemaker_session=sagemaker.Session())

#### C.) Deploy the Model

In [52]:
# Deploy the model and automatically create the endpoint configuration in the process
predictor = estimator.deploy(initial_instance_count=deploy_initial_instance_count,
                             instance_type=deploy_instance_type,
                             accelerator_type=accelerator_type,
                             serializer=serializer,
                             deserializer=deserializer,
                             model_name=model_name,
                             endpoint_name=endpoint_name,
                             data_capture_config=data_capture_config,
                             wait=True)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-26-03-54-05
INFO:sagemaker:Creating endpoint-config with name housing-endpoint
INFO:sagemaker:Creating endpoint with name housing-endpoint


------!

### Create the Data Quality Monitor

#### A.) Set the Data Quality Monitor paramaters

In [53]:
# Specify parameters
data_quality_instance_count = 1
data_quality_instance_type = 'ml.m5.xlarge'
data_quality_instance_volume_size_in_gb = 5
data_quality_max_run_time_in_seconds = 1800
data_quality_base_job_name = 'data-quality-{}'.format(model_name)
data_quality_baseline_job_name = 'data-quality-bsl-job-{}'.format(model_name)
data_quality_monitor_schedule_name = 'data-quality-mon-sch-{}'.format(model_name)
data_quality_baseline_dataset_format = DatasetFormat.csv(header=True, output_columns_position='START')

# Location in S3 where the data captured from the endpoint will be stored.
data_quality_baseline_job_output_s3_path = '{}/data-quality/baseline/'.format(model_monitor_s3_path)

# Location in S3 where the results from the monitoring job will be stored
data_quality_mon_schedule_output_s3_path = '{}/data-quality/monitoring/'.format(model_monitor_s3_path)

model_quality_base_job_name = 'model-quality-{}'.format(model_name)
model_quality_baseline_job_name_prefix = 'model-quality-bsl-job-{}'.format(model_name)
model_quality_monitor_schedule_name = 'model-quality-mon-sch-{}'.format(model_name)


In [54]:
# Create the Data Quality Monitor
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=data_quality_instance_count,
    instance_type=data_quality_instance_type,
    volume_size_in_gb=data_quality_instance_volume_size_in_gb,
    max_runtime_in_seconds=data_quality_max_run_time_in_seconds,
    base_job_name=data_quality_base_job_name,
    sagemaker_session=sagemaker_session
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


#### B.) Generate the Baseline

In [57]:
train_x_with_header_in_S3 = 's3://{}/processed/housing/train_x_with_header.csv'.format(bucket_name)

In [58]:
# Create the baseline job and generate the constraints
data_quality_monitor.suggest_baseline(
    job_name=data_quality_baseline_job_name,
    baseline_dataset=train_x_with_header_in_S3,
    dataset_format=data_quality_baseline_dataset_format,
    output_s3_uri=data_quality_baseline_job_output_s3_path,
    wait=True,
    logs=True
)

INFO:sagemaker:Creating processing-job with name data-quality-bsl-job-sagemaker-xgboost-2024-06-26-03-54-05


............2024-06-26 04:21:18.929322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-26 04:21:18.929350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-26 04:21:20.502979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-26 04:21:20.503008: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-26 04:21:20.503027: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-108-234.ec2.internal): /proc/driver/nvidia/version does not exist
2024-06-26 04:21:20.503322: I tensor

#### C.) Display the generated Data Quality Baseline

In [59]:
# Get the latest baselining job
data_quality_baseline_job = data_quality_monitor.latest_baselining_job

# Print the statistics
print('\nData Quality statistics:\n')
print(data_quality_baseline_job.baseline_statistics().body_dict['features'])

# Print the constraints
print('\nData Quality constraints:\n')
print(data_quality_baseline_job.suggested_constraints().body_dict['features'])

# Print the monitoring config
print('\nData Quality monitoring config:\n')
print(data_quality_baseline_job.suggested_constraints().body_dict['monitoring_config'])


Data Quality statistics:

[{'name': 'mssubclass', 'inferred_type': 'Fractional', 'numerical_statistics': {'common': {'num_present': 2043, 'num_missing': 0}, 'mean': 56.86735193343123, 'sum': 116180.0, 'std_dev': 41.736364920958884, 'min': 20.0, 'max': 190.0, 'distribution': {'kll': {'buckets': [{'lower_bound': 20.0, 'upper_bound': 37.0, 'count': 843.0}, {'lower_bound': 37.0, 'upper_bound': 54.0, 'count': 229.0}, {'lower_bound': 54.0, 'upper_bound': 71.0, 'count': 493.0}, {'lower_bound': 71.0, 'upper_bound': 88.0, 'count': 131.0}, {'lower_bound': 88.0, 'upper_bound': 105.0, 'count': 83.0}, {'lower_bound': 105.0, 'upper_bound': 122.0, 'count': 125.0}, {'lower_bound': 122.0, 'upper_bound': 139.0, 'count': 0.0}, {'lower_bound': 139.0, 'upper_bound': 156.0, 'count': 1.0}, {'lower_bound': 156.0, 'upper_bound': 173.0, 'count': 92.0}, {'lower_bound': 173.0, 'upper_bound': 190.0, 'count': 46.0}], 'sketch': {'parameters': {'c': 0.64, 'k': 2048.0}, 'data': [[20.0, 20.0, 60.0, 60.0, 20.0, 20.0, 1

### Schedule monitoring jobs

#### A.) Create Data Quality monitoring schedule

In [60]:
## Data Quality schedule

# Create the monitoring schedule
data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_monitor_schedule_name,
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=data_quality_mon_schedule_output_s3_path,
    statistics=data_quality_monitor.baseline_statistics(),
    constraints=data_quality_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05


#### B.) Print Schedule Details

In [61]:
## Data Quality details
# Describe and print status
data_quality_monitor_schedule_details = data_quality_monitor.describe_schedule()
print('\nData Quality Monitor - schedule details:\n')
print(data_quality_monitor_schedule_details)
print('\nData Quality Monitor - schedule status: {}'.format(data_quality_monitor_schedule_details['MonitoringScheduleStatus']))


Data Quality Monitor - schedule details:

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:453322325373:monitoring-schedule/data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05', 'MonitoringScheduleName': 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05', 'MonitoringScheduleStatus': 'Scheduled', 'MonitoringType': 'DataQuality', 'CreationTime': datetime.datetime(2024, 6, 26, 4, 25, 41, 578000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 6, 26, 4, 25, 48, 624000, tzinfo=tzlocal()), 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'}, 'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-04-25-40-962', 'MonitoringType': 'DataQuality'}, 'EndpointName': 'housing-endpoint', 'ResponseMetadata': {'RequestId': '5cf154ee-99f0-4c5c-823f-79e25285b90c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5cf154ee-99f0-4c5c-823f-79e25285b90c', 'content-type': 'application/x-amz-json-1.1', 'cont

#### C.) Start/stop/delete schedules

In [ ]:
## Data Quality schedule controls
#dq_monitor.start_monitoring_schedule()
#dq_monitor.stop_monitoring_schedule()
#dq_monitor.delete_monitoring_schedule()

### Send traffic to endpoint 

In [62]:
# Set the inference id prefix to consistently use it when invoking the endpoint
# at various steps in this notebook
inference_id_prefix = 'FromNotebook_'

In [64]:
# Dataframe of the Test Dataset to a list
df_test_x_with_header_csv_data = df_test.values.tolist()

In [65]:
# Iterate over the test data and call the predictor for each row
for index, x_test_row in enumerate(df_test_x_with_header_csv_data, start=1):
    x_test_row_string = ','.join(map(str, x_test_row))
    # Auto-generate an inference-id to track the request/response in the captured data
    inference_id = '{}{}'.format(inference_id_prefix, index)
    predicted_object = predictor.predict(data=x_test_row_string,
                                         target_model=None,
                                         target_variant=None,
                                         inference_id=inference_id)
    predicted_value = float(predicted_object[0][0])
    print('{}: Prediction for \'{}\' is {}'.format(inference_id, x_test_row_string, predicted_value))

FromNotebook_1: Prediction for '20.0,10402.0,2009.0,2009.0,0.0,0.0,0.0,1226.0,1226.0,1226.0,0.0,0.0,1226.0,0.0,0.0,2.0,0.0,3.0,1.0,6.0,0.0,2009.0,3.0,740.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0,12.200557708740234' is 128.95654296875
FromNotebook_2: Prediction for '20.0,7024.0,2005.0,2006.0,0.0,980.0,0.0,110.0,1090.0,1090.0,0.0,0.0,1090.0,1.0,0.0,1.0,1.0,2.0,1.0,5.0,0.0,2005.0,2.0,450.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,6.0,2007.0,11.964000701904297' is 158.6269073486328
FromNotebook_3: Prediction for '30.0,10914.0,1929.0,1950.0,0.0,0.0,0.0,715.0,715.0,715.0,0.0,0.0,715.0,0.0,0.0,1.0,0.0,2.0,1.0,4.0,0.0,1967.0,2.0,660.0,0.0,0.0,75.0,0.0,112.0,0.0,0.0,8.0,2006.0,12.001505851745604' is 174.23770141601562
FromNotebook_4: Prediction for '50.0,6000.0,1929.0,1950.0,0.0,0.0,0.0,862.0,862.0,950.0,208.0,0.0,1158.0,0.0,0.0,1.0,0.0,3.0,1.0,5.0,1.0,1929.0,1.0,208.0,0.0,0.0,112.0,0.0,0.0,0.0,0.0,7.0,2008.0,11.695246696472168' is 156.9562530517578
FromNotebook_5: Prediction for '50.0,8064.0,1948.0,2004.

### Analyze monitoring executions

#### A.) View monitoring executions

In [68]:
# List monitoring schedules for endpoint
print('Monitoring schedules for endpoint \'{}\':\n'.format(endpoint_name))
print(sm_client.list_monitoring_schedules(EndpointName=endpoint_name))

Monitoring schedules for endpoint 'housing-endpoint':

{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05', 'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:453322325373:monitoring-schedule/data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05', 'CreationTime': datetime.datetime(2024, 6, 26, 4, 25, 41, 578000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 6, 26, 4, 25, 48, 624000, tzinfo=tzlocal()), 'MonitoringScheduleStatus': 'Scheduled', 'EndpointName': 'housing-endpoint', 'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-04-25-40-962', 'MonitoringType': 'DataQuality'}], 'ResponseMetadata': {'RequestId': '54715a9c-5c7f-43c2-8d30-64aea5fb3289', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '54715a9c-5c7f-43c2-8d30-64aea5fb3289', 'content-type': 'application/x-amz-json-1.1', 'content-length': '523', 'date': 'Wed, 26 Jun 2024 04:31:45 GMT'}, 'RetryAttempts': 

In [69]:
## Data Quality Monitor - monitoring executions
print('Monitoring executions for schedule \'{}\':\n'.format(data_quality_monitor_schedule_name))
print(sm_client.list_monitoring_executions(MonitoringScheduleName=data_quality_monitor_schedule_name,
                                           EndpointName=predictor.endpoint_name,
                                           MonitoringTypeEquals='DataQuality',
                                           SortBy='CreationTime',
                                           SortOrder='Descending'))

Monitoring executions for schedule 'data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05':

{'MonitoringExecutionSummaries': [], 'ResponseMetadata': {'RequestId': '7e27480f-38c3-4f18-a8b5-a122e67a52dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7e27480f-38c3-4f18-a8b5-a122e67a52dc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '35', 'date': 'Wed, 26 Jun 2024 04:31:48 GMT'}, 'RetryAttempts': 0}}


#### B.) View latest constraint violations

In [70]:
## Data Quality Monitor
data_quality_constraint_violations = data_quality_monitor.latest_monitoring_constraint_violations()
if data_quality_constraint_violations != None:
    print(data_quality_constraint_violations.body_dict)

### Cleanup

In [71]:
# Delete the monitoring schedules
data_quality_monitor.delete_monitoring_schedule()

INFO:sagemaker:Deleting Monitoring Schedule with name: data-quality-mon-sch-sagemaker-xgboost-2024-06-26-03-54-05
INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2024-06-26-04-25-40-962


In [72]:
# Delete the model, endpoint configuration and endpoint
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-xgboost-2024-06-26-03-54-05
INFO:sagemaker:Deleting endpoint configuration with name: housing-endpoint
INFO:sagemaker:Deleting endpoint with name: housing-endpoint


In [77]:
# Delete all the data from the S3 buckets and delete the buckets
delete_S3_Buckets = boto3_session.resource('s3')
for bucket in delete_S3_Buckets.buckets.all():
    print(f'Deleting bucket {bucket.name}')
    
    # Delete all objects in the bucket
    bucket.objects.all().delete()
    
    # Delete the bucket itself
    bucket.delete()

print('All buckets have been deleted.')

Deleting bucket aws-athena-query-results-453322325373-us-east-1
Deleting bucket housing-dataset-8578
Deleting bucket sagemaker-automated-execution-453322325373-us-east-1
Deleting bucket sagemaker-us-east-1-453322325373
All buckets have been deleted.


### Shut down notebook resources

In [78]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [79]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>